In [ ]:
import sys
sys.path.append("..")

from models import Yolov4
from tensorflow import keras
from config import yolo_config
from utils import DataGenerator, read_annotation_lines

from glob import glob
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import os
# 使用第二張 GPU 卡
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
train_lines, val_lines = read_annotation_lines('../dataset/txt/anno_bigbox.txt', test_size=0.1)
FOLDER_PATH     = '../dataset/img_bigbox' 
class_name_path = '../class_names/classes.txt'
data_gen_train  = DataGenerator(train_lines, class_name_path, FOLDER_PATH)
data_gen_val    = DataGenerator(val_lines, class_name_path, FOLDER_PATH)

model = Yolov4(weight_path=None, class_name_path=class_name_path)

# 訓練自己的模型

In [ ]:
# class LossHistory(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.losses = []
        
#     def on_batch_end(self, batch, logs={}):
#         self.losses.append(logs.get('loss'))

# history = LossHistory()
''''''''''''''''''''''''''''''''''''''
checkpoint_filepath = '../model/20220704_big.weights'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)



epochs  = 1000
his = model.fit(data_gen_train, 
          initial_epoch=0,
          epochs=epochs, 
          val_data_gen=data_gen_val,
          callbacks=[model_checkpoint_callback],
          )

In [ ]:
x = [i for i in range(1, epochs+1)]
loss     = his.history['loss']
val_loss = his.history['val_loss']

plt.plot(x, loss, 'r')     
plt.plot(x, val_loss, 'b')     # red dotted line (no marker)
plt.legend(['loss','val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title(r'loss')
plt.show()